In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

In [2]:
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
# ## Plotly
# import plotly.offline as py
# import plotly.graph_objs as go
# py.init_notebook_mode(connected=True)
# Others
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords

from sklearn.manifold import TSNE

Using TensorFlow backend.


In [3]:
ids = pd.read_csv("scale_data/scaledata/Dennis+Schwartz/id.Dennis+Schwartz.txt", sep="[\r\n]+", header=None, names=["id"])
label_3class = pd.read_csv("scale_data/scaledata/Dennis+Schwartz/label.3class.Dennis+Schwartz.txt", sep="[\r\n]+", names=["3class_label"])
label_4class = pd.read_csv("scale_data/scaledata/Dennis+Schwartz/label.4class.Dennis+Schwartz.txt", sep="[\r\n]+",names=["4class_label"])
rating = pd.read_csv("scale_data/scaledata/Dennis+Schwartz/rating.Dennis+Schwartz.txt", sep="[\r\n]+",names=["rating"])
subj = pd.read_csv("scale_data/scaledata/Dennis+Schwartz/subj.Dennis+Schwartz.txt", sep="[\r\n]+",names=["subj_extraction"])

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support 

In [4]:
dennis = pd.concat([ids,label_3class, label_4class, rating, subj], axis = 1)
dennis.head(5)

,id,3class_label,4class_label,rating,subj_extraction
0,29420,0,0,0.1,"in my opinion , a movie reviewer's most import..."
1,17219,0,0,0.2,"you can watch this movie , that is based on a ..."
2,18406,0,0,0.2,"this is asking a lot to believe , and though i..."
3,18648,0,0,0.2,no heroes and no story are the main attributes...
4,20021,0,0,0.2,"this is not an art movie , yet i saw it an art..."


In [5]:
dennis["strongly neg"]=dennis.apply(lambda x:x["4class_label"]==0, axis=1)
dennis["neg"]=dennis.apply(lambda x:x["4class_label"]==1, axis=1)
dennis["pos"]=dennis.apply(lambda x:x["4class_label"]==2, axis=1)
dennis["strongly pos"]=dennis.apply(lambda x:x["4class_label"]==3, axis=1)

In [12]:
y_d = dennis['3class_label']
dennis['n']=dennis.apply(lambda x:x["4class_label"]==0, axis=1)
dennis["neg"]=dennis.apply(lambda x:x["4class_label"]==1, axis=1)
dennis["pos"]=dennis.apply(lambda x:x["4class_label"]==2, axis=1)
y_d_encode = dennis[['n', 'neg', 'pos']]

In [16]:
from sklearn.model_selection import train_test_split
xtrain_d, xvalid_d, ytrain_d, yvalid_d = train_test_split(list(dennis['subj_extraction']), y_d, 
                                                  stratify=y_d, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [13]:
xtrain_d, xvalid_d, ytrain_d_encode, yvalid_d_encode = train_test_split(dennis['subj_extraction'], y_d_encode, 
                                                  stratify=y_d, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [182]:
# we use x_train as docs, y_d as labels
print (ytrain_d.shape)
print (ytrain_d_encode.shape)
print(len(xtrain_d[1].split()))

(924,)
(924, 3)
322


In [58]:
def x(c,f):
    return c,f

In [69]:
docs = ['having a dark humor about it that is funnier than hell', 'the american title to the film , boiling point , is misleading : in japanese the title refers in slang to a baseball score', 'she is going to the library', 
      'it is a call for artistic freedom']
def padded_doc(docs):
    t = Tokenizer()
    t.fit_on_texts(docs)
    # integer encode the documents
    encoded_docs = t.texts_to_sequences(docs)
    # pad documents to a max length of 4 words
    max_length = 10
    padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
    # print(padded_docs)
    return padded_docs
padded_doc(docs)
t = Tokenizer()
t.fit_on_texts(docs)
print (t.texts_to_sequences(docs))
# p_d = padded_doc(docs)
# gd.get_glove_m(p_d, )
vocab_size = len(t.word_index) + 1
print(vocab_size)
print (t.word_index)
print (padded_doc(docs).shape)

[[8, 3, 9, 10, 11, 5, 12, 1, 13, 14, 15], [2, 16, 6, 4, 2, 17, 18, 19, 1, 20, 7, 21, 2, 6, 22, 7, 23, 4, 3, 24, 25], [26, 1, 27, 4, 2, 28], [5, 1, 3, 29, 30, 31, 32]]
33
{'is': 1, 'the': 2, 'a': 3, 'to': 4, 'it': 5, 'title': 6, 'in': 7, 'having': 8, 'dark': 9, 'humor': 10, 'about': 11, 'that': 12, 'funnier': 13, 'than': 14, 'hell': 15, 'american': 16, 'film': 17, 'boiling': 18, 'point': 19, 'misleading': 20, 'japanese': 21, 'refers': 22, 'slang': 23, 'baseball': 24, 'score': 25, 'she': 26, 'going': 27, 'library': 28, 'call': 29, 'for': 30, 'artistic': 31, 'freedom': 32}
(4, 10)


In [60]:
y,f = x(7,8)

In [61]:
y

7

In [14]:
import glove_dl21 as gd

In [85]:
vocab_size, embedding_matrix, padded_docs = gd.get_glove_m(xtrain_d, ytrain_d)

In [86]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.038194  , -0.24487001,  0.72812003, ..., -0.1459    ,
         0.82779998,  0.27061999],
       [-0.27085999,  0.044006  , -0.02026   , ..., -0.4923    ,
         0.63687003,  0.23642001],
       ..., 
       [ 0.49579999, -0.41578001,  0.19389001, ..., -0.30579001,
         0.45275   ,  0.76160002],
       [-0.46489999, -1.06060004, -0.10805   , ...,  0.036301  ,
        -0.85459   ,  0.29347   ],
       [-0.032544  ,  0.093303  ,  0.0076568 , ...,  0.63237   ,
         0.43331   ,  0.84724998]])

In [87]:
x_test_padded_docs = gd.padded_doc(xvalid_d)


In [88]:
print (embedding_matrix.shape)
print (vocab_size)
padded_docs.shape


(17911, 100)
17911


(924, 400)

## cnn 

In [91]:

from keras.layers import Dense, Dropout, Flatten, Input, MaxPooling1D, Convolution1D, Embedding
from keras.layers.merge import Concatenate
from keras.models import Sequential, Model
def build_model(embedding_matrix, vocab_size):
    # define model
    embedding_dim = 50
    filter_sizes = (2, 4)
    num_filters = 10
    dropout_prob = (0.5, 0.8)
    hidden_dims = 50

    # Build model
    sequence_length = 400
    input_shape = (sequence_length,)

    model_input = Input(shape=input_shape)

    # Static model does not have embedding layer

    z = Embedding(vocab_size, 100, input_length=sequence_length,weights=[embedding_matrix], name="embedding")(model_input)

    z = Dropout(dropout_prob[0])(z)

    # Convolutional block
    conv_blocks = []
    for sz in filter_sizes:
        conv = Convolution1D(filters=num_filters,
                             kernel_size=sz,
                             padding="valid",
                             activation="relu",
                             strides=1)(z)
        conv = MaxPooling1D(pool_size=1)(conv)
        conv = Flatten()(conv)
        conv_blocks.append(conv)
    z = Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]

    z = Dropout(dropout_prob[1])(z)
    z = Dense(hidden_dims, activation="relu")(z)
    model_output = Dense(3, activation="sigmoid")(z)

    model = Model(model_input, model_output)
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

    weights = np.array([v for v in embedding_matrix])
    print("Initializing embedding layer with glove weights, shape", weights.shape)
    embedding_layer = model.get_layer("embedding")
    embedding_layer.set_weights([weights])
    return model

# # Initialize weights with word2vec
# if model_type == "CNN-non-static":
#     weights = np.array([v for v in embedding_weights.values()])
#     print("Initializing embedding layer with word2vec weights, shape", weights.shape)
#     embedding_layer = model.get_layer("embedding")
#     embedding_layer.set_weights([weights])

# Train the model
model.fit(padded_docs, ytrain_d_encode, epochs=50, verbose=1)

Epoch 1/50
924/924 [==============================] - 5s 5ms/step - loss: 0.2802 - acc: 0.7868
Epoch 2/50
924/924 [==============================] - 4s 5ms/step - loss: 0.2553 - acc: 0.8074
Epoch 3/50
924/924 [==============================] - 4s 4ms/step - loss: 0.2705 - acc: 0.8128
Epoch 4/50
924/924 [==============================] - 4s 4ms/step - loss: 0.2530 - acc: 0.8052
Epoch 5/50
924/924 [==============================] - 4s 4ms/step - loss: 0.2502 - acc: 0.8225
Epoch 6/50
924/924 [==============================] - 4s 4ms/step - loss: 0.2546 - acc: 0.8193
Epoch 7/50
924/924 [==============================] - 4s 5ms/step - loss: 0.2211 - acc: 0.8214
Epoch 8/50
924/924 [==============================] - 4s 5ms/step - loss: 0.2184 - acc: 0.8225
Epoch 9/50
924/924 [==============================] - 4s 5ms/step - loss: 0.2363 - acc: 0.8160
Epoch 10/50
924/924 [==============================] - 4s 5ms/step - loss: 0.2018 - acc: 0.8398
Epoch 11/50
924/924 [============================

In [92]:
model = build_model(embedding_matrix, vocab_size)
model.fit(padded_docs[:100], ytrain_d_encode[:100], epochs=50, verbose=1)

Initializing embedding layer with glove weights, shape (17911, 100)
Epoch 1/50
100/100 [==============================] - 2s 22ms/step - loss: 1.2077 - acc: 0.4400
Epoch 2/50
100/100 [==============================] - 1s 6ms/step - loss: 0.9949 - acc: 0.4700
Epoch 3/50
100/100 [==============================] - 1s 6ms/step - loss: 0.9172 - acc: 0.4900
Epoch 4/50
100/100 [==============================] - 1s 6ms/step - loss: 0.8941 - acc: 0.4900
Epoch 5/50
100/100 [==============================] - 1s 6ms/step - loss: 0.8334 - acc: 0.4900
Epoch 6/50
100/100 [==============================] - 1s 7ms/step - loss: 0.8868 - acc: 0.4800
Epoch 7/50
100/100 [==============================] - 1s 7ms/step - loss: 0.7795 - acc: 0.5000
Epoch 8/50
100/100 [==============================] - 1s 7ms/step - loss: 0.6845 - acc: 0.4900
Epoch 9/50
100/100 [==============================] - 1s 6ms/step - loss: 0.7949 - acc: 0.4900
Epoch 10/50
100/100 [==============================] - 1s 6ms/step - loss: 0

In [23]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 400)          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 400, 100)     1791100     input_2[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 400, 100)     0           embedding[0][0]                  
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 399, 10)      2010        dropout_1[0][0]                  
__________________________________________________________________________________________________
conv1d_2 (

## visuliaze the prediction

In [108]:
# evaluate the model
loss, accuracy = model.evaluate(x_test_padded_docs, yvalid_d, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 44.660194


In [27]:
xtrain_d_glv = np.array(xtrain_d_glv)
xvalid_d_glv = np.array(xvalid_d_glv)
xtrain_d_glv.shape

(924, 300)

In [31]:
num_max = 300
def model_3_layer():
    model = Sequential()
    model.add(Dense(512, activation = 'relu', input_shape=(num_max,)))
    model.add(Dropout(0.2))
    model.add(Dense(256, activation = 'relu'))
    model.add(Dropout(0.2))
    model.add(Dense(3, activation = 'sigmoid'))
    model.summary()
#     model.compile(optimizer='rmsprop', 
#                   loss='categorical_crossentropy',
#                   metrics=['accuracy'])
    model.compile(optimizer='rmsprop', 
              loss='binary_crossentropy',
              metrics=['accuracy'])
    print ('compile done')
    return model



In [34]:
# serialize model to JSON
model_json = model.to_json()
with open("model_gl.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_gl.h5")
print("Saved model to disk")

Saved model to disk


In [24]:
# serialize model to JSON
model_cnn_json = model.to_json()
with open("model_gl_cnn.json", "w") as json_file:
    json_file.write(model_cnn_json)
# serialize weights to HDF5
model.save_weights("model_gl_cnn.h5")
print("Saved model to disk")

Saved model to disk


# glove_nn on the other writer

In [25]:
ids = pd.read_csv("scale_data/scaledata/James+Berardinelli/id.James+Berardinelli.txt", sep="[\r\n]+", header=None, names=["id"])
# label_3class = pd.read_csv("scale_data/scaledata/James+Berardinelli/label.3class.James+Berardinelli.txt", sep="[\r\n]+", names=["3class_label"])
label_3class = pd.read_csv("scale_data/scaledata/James+Berardinelli/label.3class.James+Berardinelli.txt", sep="[\r\n]+",names=["3class_label"])

label_4class = pd.read_csv("scale_data/scaledata/James+Berardinelli/label.4class.James+Berardinelli.txt", sep="[\r\n]+",names=["4class_label"])
rating = pd.read_csv("scale_data/scaledata/James+Berardinelli/rating.James+Berardinelli.txt", sep="[\r\n]+",names=["rating"])
subj = pd.read_csv("scale_data/scaledata/James+Berardinelli/subj.James+Berardinelli.txt", sep="[\r\n]+",names=["subj_extraction"])

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: ParserWarnin

In [26]:
james = pd.concat([ids,label_3class, label_4class, rating, subj], axis = 1)


In [27]:
james["strongly neg"]=james.apply(lambda x:x["4class_label"]==0, axis=1)
james["neg"]=james.apply(lambda x:x["4class_label"]==1, axis=1)
james["pos"]=james.apply(lambda x:x["4class_label"]==2, axis=1)
james["strongly pos"]=james.apply(lambda x:x["4class_label"]==3, axis=1)

In [28]:
y_d = dennis['3class_label']
james['n']=james.apply(lambda x:x["4class_label"]==0, axis=1)
james["neg"]=james.apply(lambda x:x["4class_label"]==1, axis=1)
james["pos"]=james.apply(lambda x:x["4class_label"]==2, axis=1)
y_j_encode = james[['n', 'neg', 'pos']]

In [29]:
x_j = james.subj_extraction.values
y_j = james['3class_label']


In [30]:
from sklearn.model_selection import train_test_split
xtrain_j, xvalid_j, ytrain_j, yvalid_j = train_test_split(x_j, y_j, 
                                                  stratify=y_j, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [31]:
xtrain_j, xvalid_j, ytrain_j_encode, yvalid_j_encode = train_test_split(x_j, y_j_encode, 
                                                  stratify=y_j, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [60]:
xtrain_j_glv = [g.sentence_vectorize(sent) for sent in xtrain_j]
xvalid_j_glv = [g.sentence_vectorize(sent) for sent in xvalid_j]
xtrain_j_glv = np.array(xtrain_j_glv)
xvalid_j_glv = np.array(xvalid_j_glv)

xtrain_j_glv.shape

(1176, 300)

In [33]:
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
import numpy
import os

In [77]:
xvalid_j_padded = gd.padded_doc(xvalid_j)
xvalid_j_padded.shape
xtrain_j_padded = gd.padded_doc(xtrain_j)
xtrain_j_padded.shape
# xtrain_j_padded = xtrain_j_padded[:924,:]
# yvalid_j_encode.shape

(1176, 400)

## load the pre_trained model and evaluate on james directly

In [47]:

# load json and create model
json_file = open('model_gl_cnn.json', 'r')
loaded_model_cnn_json = json_file.read()
json_file.close()
loaded_model_cnn = model_from_json(loaded_model_cnn_json)
# load weights into new model
loaded_model_cnn.load_weights("model_gl_cnn.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model_cnn.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model_cnn.evaluate(xvalid_j_padded, yvalid_j_encode, verbose=1)
print("%s: %.2f%%" % (loaded_model_cnn.metrics_names[1], score[1]*100))
# loaded_model.summary()


Loaded model from disk
131/131 [==============================] - 0s 3ms/step
acc: 69.72%


In [57]:
xvalid_j_padded.shape

(131, 400)

In [82]:
vocab_size_j, embedding_matrix_j, padded_docs_j = gd.get_glove_m(xtrain_j, ytrain_j)

In [ ]:
model = build_model(embedding_matrix_j, vocab_size_j)

In [ ]:
model.load

## fine tuning

In [73]:
for layer in model.layers[:1]:
    layer.trainable = False

In [74]:
loaded_model_cnn

In [78]:
from keras import optimizers

loaded_model_cnn.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [81]:
loaded_model_cnn.fit(xtrain_j_padded[:924,:], ytrain_j_encode[:924], epochs=50, verbose=1)

Epoch 1/50


InvalidArgumentError: indices[7,188] = 18469 is not in [0, 17911)
	 [[Node: embedding_9/Gather = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, validate_indices=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](embedding_9/embeddings/read, embedding_9/Cast)]]

Caused by op 'embedding_9/Gather', defined at:
  File "/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    ap

## cnn on glove

In [73]:
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model

In [82]:
def build_1d_cnn():
    model = Sequential()
    model.add(Conv1D(128,5,
                 activation='relu'))
    model.add(MaxPooling1D(5))
    model.add(Conv2D(64, 5, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    return model
    

In [ ]:
# set parameters:
max_features = 5000
maxlen = 300
batch_size = 32
embedding_dims = 50
filters = 100
kernel_size = 3
hidden_dims = 250
epochs = 2


model = Sequential()
model.add(Embedding(5000,
                    400,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(3))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(xtrain_j_glv, ytrain_j_encode,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(xvalid_j_glv, yvalid_j_encode))